In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

data = pd.read_csv("dataset/fifadata.csv")
data.head(20)

FileNotFoundError: [Errno 2] File b'dataset/fifadata.csv' does not exist: b'dataset/fifadata.csv'

In [ ]:
from IPython.display import display

def describe(data: pd.core.frame.DataFrame):
    start = 0;
    end= start + 10;
    while end <= data.columns.size and start < data.columns.size:
        display(f"Data from {start+1} to {end}")
        data1 = data.iloc[:,start:end]
        display(data1.head())
        #display(data1.describe())
        display("======================================================================================================")
        start=end;
        end=start+10;
        if (end >= data.columns.size):
            end=data.columns.size


In [ ]:
noFeatureColumns = ['Unnamed: 0','ID','Photo','Flag','Name','Club Logo','Real Face','Loaned From','Jersey Number','Preferred Foot','Contract Valid Until']
dataWithPredictableFeature = data.drop(noFeatureColumns,axis=1)



In [ ]:
plt.figure(figsize=(20,10))
missing_data = dataWithPredictableFeature.isnull().sum().sort_values(ascending=False)
missing_data.plot.bar()

In [ ]:
trash_data = dataWithPredictableFeature[dataWithPredictableFeature['RDM'].isnull()].index
dataWithoutMissingVal = dataWithPredictableFeature.drop(trash_data)

In [ ]:
dataWithoutMissingVal = dataWithPredictableFeature.drop(trash_data)

In [ ]:
plt.figure(figsize=(20,10))
missing_data_1 = dataWithoutMissingVal.isnull().sum().sort_values(ascending=False)
missing_data_1.plot.bar()

In [ ]:
dataWithoutMissingVal.reset_index(drop=True,inplace=True)
dataWithoutMissingVal



In [ ]:
dataWithoutMissingVal = dataWithoutMissingVal.drop(dataWithoutMissingVal[dataWithoutMissingVal['Wage'] == '€0'].index)
dataWithoutMissingVal = dataWithoutMissingVal.drop(dataWithoutMissingVal[dataWithoutMissingVal['Value'] == '€0'].index)
plt.figure(figsize=(20,10))
missing_data_2 = dataWithoutMissingVal.isnull().sum().sort_values(ascending=False)
missing_data_2.plot.bar()

In [ ]:
splited = dataWithoutMissingVal["Joined"].str.split(",", n = 1, expand = True)   
dataWithoutMissingVal['monthJoined'] = splited[0]
dataWithoutMissingVal['yearJoined'] = splited[1].astype('float')

In [ ]:
yearJoined = (int) (dataWithoutMissingVal['yearJoined'].mean())
dataWithoutMissingVal['yearJoined']  = dataWithoutMissingVal['yearJoined'].fillna(yearJoined)
dataWithoutMissingVal = dataWithoutMissingVal.drop(['monthJoined','Joined'],axis=1)

In [ ]:
dataWithoutMissingVal = dataWithoutMissingVal.drop(dataWithoutMissingVal[dataWithoutMissingVal['Wage'] == '€0'].index)
plt.figure(figsize=(20,10))
missing_data_3 = dataWithoutMissingVal.isnull().sum().sort_values(ascending=False)
missing_data_3.plot.bar()

# Extract information from dataset
## Change height info from foot/inches to cm (convert from categorical to numeric)


In [ ]:
#df = data
data_processing = dataWithoutMissingVal
data_processing['Height_Foot'], data_processing['Height_Inch'] = data_processing['Height'].str.split("'", 1).str
data_processing['Height_Foot'] = data_processing['Height_Foot'].astype('double')
data_processing['Height_Inch'] = data_processing['Height_Inch'].astype('double')
data_processing['Height_cm'] = (data_processing['Height_Foot'] * 12 + data_processing['Height_Inch']) * 2.54

In [ ]:
data_processing = data_processing.drop(['Height','Height_Foot','Height_Inch'],axis=1)

In [ ]:
data_processing['Weight']

In [ ]:
describe(data)

In [ ]:
weight = data_processing['Weight'].str.extract(r'(\d+)')
data_processing['Weight_lbs'] = weight.astype('float')
data_processing = data_processing.drop(['Weight'],axis=1)
data_processing['df_Newfeature'] = (data_processing['StandingTackle'] + data_processing['SlidingTackle']) /2
data_processing = data_processing.drop(['StandingTackle','SlidingTackle'],axis=1)

In [ ]:
describe(data_processing)

In [ ]:
def eval_data(data,columns):
    for colName in columns:
        col_Val = colName + "_val"
        data[col_Val] = data[colName].apply(lambda x: eval(x))
        

In [ ]:
list_need_eval_columns = ['LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM'
                     ,'CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB'
                     ,'CB','RCB','RB']
eval_data(data_processing,list_need_eval_columns)



In [ ]:
list_after_eval = [data + "_val" for data in list_need_eval_columns]
data_processing = data_processing.drop(list_need_eval_columns,axis=1)

In [ ]:
describe(data_processing)

In [ ]:
data_processing["Release Clause"]

In [ ]:
#Bao helped writing this code to solve a problem with the calculation of the money value
import re
import math
def calculate_money(money):
    if isinstance(money,str):
        if (money is None or len(money) <= 0):
            return (float)(0)
    elif math.isnan(money):
        return (float)(0)
    base = 1
    last = money[-1].casefold() 
        
    if "m".casefold() == last:
        base = 1000000
    elif "k".casefold() == last:
        base = 1000
    
    if base > 1:
        val = (float) (money[1:-1])
    else:
        val = (float) (money[1:])
    return val*base



In [ ]:
data_processing['Value_val'] = data_processing['Value'].apply(calculate_money)
data_processing['Wage_val'] = data_processing['Wage'].apply(calculate_money)
data_processing['euro_val'] = data_processing['Release Clause'].apply(calculate_money)
data_processing = data_processing.drop(['Value','Wage','Release Clause'],axis=1)

In [ ]:
data_processing.head()

In [ ]:
plt.figure(figsize=(20,10))
missing_data_4 = data_processing.isnull().sum().sort_values(ascending=False)
missing_data_4.plot.bar()

In [ ]:
data_processing['Weight_lbs'] = data_processing['Weight_lbs'].astype('float')


In [ ]:
data_processing.select_dtypes("object")

In [ ]:
data_processing.head(5)

# After cleaning data and transform into correct data form we want to perform encoding categorical before performing machine learning algorithm

In [ ]:
#combinding the Musclememory stats for less numbers and removing them
musclememory=['LongShots','Aggression','Penalties','Composure','Marking','Interceptions','Positioning','Vision','Curve','FKAccuracy','LongPassing','BallControl','Reactions','Balance','ShotPower','Crossing','Finishing','HeadingAccuracy','ShortPassing','Volleys','Dribbling']    

data_processing["musclememory"] = data_processing[musclememory].mean(axis=1)
data_processing= data_processing.drop(musclememory,axis=1)




In [ ]:
#combinding the Goalkeeper stats for less numbers and removing them
goalkeeper=["GKDiving","GKHandling","GKKicking","GKPositioning","GKReflexes"]
data_processing["Goalkeeper"] = data_processing[goalkeeper].mean(axis=1)
data_processing = data_processing.drop(goalkeeper,axis=1)

In [ ]:
#combinding the Musclestrength stats for less numbers and removing them
musclestrength=['Jumping','Stamina','Strength','Acceleration','SprintSpeed','Agility']    
data_processing["Musclestrength"] = data_processing[musclestrength].mean(axis=1)
data_processing = data_processing.drop(musclestrength,axis=1)
df = pd.get_dummies(data_processing)




In [ ]:
position =[]
for col in list_need_eval_columns:
    colName = col + "_val"
    position.append(colName)

data_processing["Positionoftheplayer"] = data_processing[position].mean(axis=1)
df = pd.get_dummies(data_processing)
df.head(5)


data_processing = data_processing.drop(position,axis=1)






    




In [ ]:
#these stats are similar to eachother so they where added together also
Specialstats=['International Reputation','Weak Foot','Skill Moves']
    
data_processing["SpecialStats"] = data_processing[Specialstats].mean(axis=1)
data_processing = data_processing.drop(Specialstats,axis=1)
df = pd.get_dummies(data_processing)

In [ ]:
#Stats that are not affecting the value of the player
Dumpstats=['yearJoined','Position','Body Type',"Work Rate","Club","Nationality","Height_cm","Weight_lbs"]
    
data_processing = data_processing.drop(Dumpstats,axis=1)
df = pd.get_dummies(data_processing)

In [ ]:
describe(data_processing)

# Data preprocessing is done. Now we can try to run some ML algorithm
## We first split our dataset into train and test

In [ ]:
X = df.drop(['Value_val'],axis=1)
y = df['Value_val']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=4000, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [ ]:
import sklearn.metrics as metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
describe(X)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

tree = 20
tree_numbers,MAE,MSE,RMSE = [],[],[],[]
while tree <= 200:
    regressor = RandomForestRegressor(n_estimators=tree, random_state=0)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    tree_numbers.append(str(tree))
    MAE.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    tree+=10

In [ ]:
plt.style.use('seaborn')
plt.plot(tree_numbers, MAE, label = 'Mean absolute error')
#plt.plot(tree_numbers, MSE, label = 'Mean square error')
#plt.plot(tree_numbers, RMSE, label = 'Root mean square error')

In [ ]:
#plt.matshow(data_processing.corr())
#plt.show()
plt.figure(figsize=(20,20))
mySubset = data_processing.iloc[:,40:]
corr = data_processing.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,cmap="coolwarm")

In [ ]:
goal_corr = corr.sort_values(by=['Value_val'], ascending=False).head(1)

In [ ]:
plt.figure(figsize=(20,4))
goal_corr = corr.sort_values(by=['Value_val'], ascending=False).head(1)
sns.heatmap(goal_corr, 
        cmap="Blues")

In [ ]:
X = df.drop(['Value_val','Wage_val'],axis=1)
y = df['Value_val']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

def runRandomForest(X,y,tree_numbers):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    regressor = RandomForestRegressor(n_estimators=tree_numbers, random_state=0)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
df.columns
X = df.drop(['Value_val'],axis=1)
y = df['Value_val']
runRandomForest(X,y,70)

In [ ]:
data_processing.select_dtypes(include='object')

In [ ]:
my_df = pd.get_dummies(data_processing.drop(['Club'],axis=1))
X1 = df.drop(['Wage_val'],axis=1)
y1 = df['Wage_val']
runRandomForest(X1,y,70)

In [ ]:
Wage = my_df['Wage_val']
sns.distplot(Wage)

In [ ]:
describe(data)